# Solutions — Exercices (Rectangle, Membres, Blackjack)

## Exercice 1 — Classe `Rectangle`

Définir une classe `Rectangle` avec :
- constructeur avec valeurs par défaut pour longueur et largeur
- attribut `nom = 'rectangle'`
- méthode d'affichage des attributs
- méthode `surface()` qui renvoie la surface


In [1]:
class Rectangle: 
    nom  = "rectangle"
    
    def __init__(self,longueur = 1 , largeur = 2):
        self.longeut = float(longueur)
        self.largeur = float(largeur)
        
    def Affichage(self) :
        print(f"Nom : {self.nom} ;longueur : {self.longueur} ,largeur : {self.largeur}" )
    
    def surface(self):
        return self.longueur * self.largeur

         

## Exercice 2 — Membre, Admin, Utilisateur

- Un membre a : username, nom, date_inscription, date_expiration
- Compte actif pendant une durée donnée (par défaut 365 jours, modifiable par l'administration)
- Admin : compte n'expire pas, password, peut activer/désactiver comptes et mettre à jour l'état selon la date
- Utilisateur hérite de Membre et peut `renouveler(days)`

Ensuite :
- Instancier un membre et afficher sa date d'inscription
- Instancier un admin
- Instancier user1 et user2
- L'admin désactive le compte de user2
- user1 renouvelle son compte de 40 jours puis afficher la date d'expiration


In [1]:
from datetime import datetime, timedelta
################################################################################################
class Membre:
     jour_gratuits = 365
     def __init__(self, username, nom,date_inscription=None):
        self.username = username
        self.nom = nom
        self.date_inscription = datetime.now()
        self.date_expiration = self.date_inscription + timedelta(days=self.jour_gratuits)
        self.active = True
        
     def afficher_date_inscription(self):   
             print(f"date d'inscription est : {self.date_inscription}")
     
     def afficher_date_expiration(self):  
          print(f"date d'inscription est : {self.date_expiration}")
    
     def is_active(self):
         return self.active and datetime.now() <= self.date_expiration
     
     
 ######################################################################################################   
class Admin(Membre):
    
      def __init__(self, username, nom, password, date_inscription=None):
             super().__init__(username, nom, date_inscription)
             self.password = password
              # Admin  compte n'expire pas
             self.date_expiration = None
             self.active = True
             
               
      def mettre_a_jour_etat_comptes(liste_membres):
            for mb in liste_membres :
                 if isinstance(mb,Membre) and mb.date_expiration is not None :
                    if datetime.now() > mb.date_expiration:
                         mb.active = False
                         
                
                
                
                
      def activer_compte(self, membre):
          membre.active = True
          if membre.date_expiration is None or datetime.now() > membre.date_expiration:
                   membre.date_expiration = datetime.now() + timedelta(days=Membre.jour_gratuits)
                    
      def desactiver_compte(self, membre):
                         membre.active = False
##############################################################################################################

class User(Membre):
     def __init__(self, username, nom, date_inscription=None):
           super().__init__(username, nom, date_inscription)
     
     def renouvler(self,days):
          
       if self.date_expiration is None or self.date_expiration < datetime.now() :
               self.date_expiration = datetime.now() + timedelta(days=days)
       else:
                 self.date_expiration = self.date_expiration + timedelta(days=days)              

       self.active = True   # pour garantie que le compte rah active 
     
     
m1 = Membre("fares","samer")
m1.afficher_date_inscription()
m1.afficher_date_expiration()

A =Admin("bacha","sihem","sb193")

user1 = User("meriem","oberk")
user2 = User("amin","yassir")
print(user2.active)

A.desactiver_compte(user2)
print(user2.active)
user1.afficher_date_expiration()
user1.renouvler(40)
user1.afficher_date_expiration()

date d'inscription est : 2025-11-26 22:57:12.713511
date d'inscription est : 2026-11-26 22:57:12.713511
True
False
date d'inscription est : 2026-11-26 22:57:12.714516
date d'inscription est : 2027-01-05 22:57:12.714516


## Exercice 3 — Blackjack (modélisation OO)

Implémentation d'un jeu de Blackjack simplifié avec :
- cartes, deck, mains, règles pour As (1 ou 11), Blackjack naturel, bust
- distribution initiale: 2 cartes joueur visibles, 2 cartes croupier (1 cachée)
- croupier tire jusqu'à au moins 17
- gains : Blackjack -> 3x mise, victoire normale -> 2x mise

On montre une simulation automatique (stratégie simple) pour une partie.


In [5]:
import random

class Card :
    def __init__(self,rank,suit):
        self.rank = rank
        self.suit = suit 
       
    def value(self):
        if self.rank in ["J", "Q", "K"]:
            return 10
        if self.rank == "A":
           return 11 
        return int(self.rank)
    
    def __str__(self):
        return f"{self.rank} {self.suit}" # truturné les info ta3 l card
    
############################################################################################
class Deck:
    def __init__(self):
        suits = ["♠", "♥", "♦", "♣"]
        ranks = ["A", "2", "3", "4", "5", "6", "7", "8", "9", "10", "J", "Q", "K"]
        self.cards = [Card(rank, suit) for suit in suits for rank in ranks]
        random.shuffle(self.cards)

    def draw(self):
        return self.cards.pop()
#################################################################################################
class Hand:
    def __init__(self):
        self.cards = []

    def ajouté_card(self, card):
        self.cards.append(card)

    def total(self):
        total = sum(card.value() for card in self.cards)
        aces = sum(1 for c in self.cards if c.rank == "A")

        while total > 21 and aces:
            total -= 10
            aces -= 1

        return total

    def is_blackjack(self):
        return len(self.cards) == 2 and self.total() == 21

    def is_bust(self):
        return self.total() > 21

################################################################################    
    
class Player:
    def __init__(self, name):
        self.name = name
        self.hand = Hand()

    def hit(self, deck):
        self.hand.ajouté_card(deck.draw())
        
        
######################################################################################
class Dealer(Player):
    def play(self, deck):
        while self.hand.total() < 17:
            self.hit(deck)
##################################################################################""""""""""
class Game:
    def __init__(self, bet=10):
        self.bet = bet
        self.deck = Deck()
        self.player = Player("Fares")
        self.dealer = Dealer("Croupier")

    def start(self):
        # Distribution initiale
        self.player.hit(self.deck)
        self.player.hit(self.deck)
        self.dealer.hit(self.deck)
        self.dealer.hit(self.deck)

        print(f"Player cards: {[str(c) for c in self.player.hand.cards]}")
        print(f"Dealer visible card: {self.dealer.hand.cards[0]}")

        # Vérifier blackjack
        if self.player.hand.is_blackjack():
            print("Blackjack du joueur! Gain = 3x la mise")
            return

        # Tour du joueur
        while True:
            choice = input("Hit or Stand? ").lower()
            if choice == "hit":
                self.player.hit(self.deck)
                print("Player cards:", [str(c) for c in self.player.hand.cards])
                
                if self.player.hand.is_bust():
                    print("Player bust! You lose.")
                    return
            else:
                break

        # Tour du croupier
        self.dealer.play(self.deck)
        print("Dealer cards:", [str(c) for c in self.dealer.hand.cards])

        if self.dealer.hand.is_bust():
            print("Dealer bust! Player wins.")
            return

        # Comparaison
        p = self.player.hand.total()
        d = self.dealer.hand.total()

        print(f"Player total = {p}, Dealer total = {d}")

        if p > d:
            print("Player wins! Gain = 2x la mise")
        elif p < d:
            print("Dealer wins!")
        else:
            print("Égalité, mise rendue.")
            
print( "jaha")


game = Game()
game.start()            


jaha
Player cards: ['7 ♦', 'K ♣']
Dealer visible card: J ♥
Dealer cards: ['J ♥', 'A ♥']
Player total = 17, Dealer total = 21
Dealer wins!
